# Sentinel 2 Cloudless Mosaic

This tutorial constructs a *cloudless mosaic* (also known as a composite) from a time series of [Sentinel-2 Level-2A](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a) images and is modified from the example notebook provided by Microsoft. This notebook performs the following steps:

* Find a time series of images within a bounding box
* Stack those images together into a single array
* Mask clouds and cloud shadows
* Synthesize a panchromatic band by averageing Red, Green, Blue and NIR bands
* Compute the cloudless mosaic by taking a median
* Save the result to a GeoTiff

This notebook is designed for the processing of large areas, so tasks like plotting that are useful but resource-intensive are omitted

## Setup

In [1]:
import numpy as np
import xarray as xr
import pandas as pd

import rasterio.features
import rioxarray
import stackstac
import pystac_client
import planetary_computer

import pyproj
from shapely.ops import transform
from shapely.geometry import Polygon

import xrspatial.multispectral as ms

from dask_gateway import GatewayCluster
from dask import visualize

## Create a Dask cluster

We're going to process a large amount of data. To cut down on the execution time, we'll use a Dask cluster to do the computation in parallel, adaptively scaling to add and remove workers as needed. See [Scale With Dask](../quickstarts/scale-with-dask.ipynb) for more on using Dask.

In [2]:
cluster = GatewayCluster()  # Creates the Dask Scheduler. Might take a minute.

client = cluster.get_client()

cluster.adapt(minimum=4, maximum=24)
print(cluster.dashboard_link)

https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.9ff4ca93053d4fb5ab262534d1c88a43/status


## Discover data

In this step we define our bounding box by creating a Shapely Polygon object. The Polygon object is created from a set of coordinate pairs in **Latitude and Longitude** (epsg 3857). A simple way of getting the coordinate pairs is by creating a bounding box in Google Earth, saving it to a kml, then opening it as a text file and copying the coordinates.

At this point, you'll have to decide if you want to process multiple years at once, or if you want to process the years separately. This decision comes down to how much compute power you have access to. For the Dask cluster parameters specified, `cluster.adapt(minimum=4, maximum=24)`, the maximum amount of images used should be less than 200. You can alter the number of images used by changing the values of `date_range`, `max_cloud`, and `pol`.

In [3]:
# date range
year = 2021
date_range = f"{year}-01-01/{year+1}-01-01"

# date_range = "2016-01-01/2022-01-01"

# max cloud coverage
max_cloud = 10

# possible bounding boxes
poroa =[[175.9540924324072,-39.79467075565404],
        [175.945114873355,-39.5925815994339],
        [175.7466289836702,-39.59528350422168],
        [175.7518945072953,-39.79792833072585],
        [175.9540924324072,-39.79467075565404]]

study_area = [[174.3195824446477,-39.31681541495579],
              [174.2992147571175,-40.1298100050477],
              [176.5561381739111,-40.14413275288879],
              [176.5430092169263,-39.32607797703585],
              [174.3195824446477,-39.31681541495579]] 

# create shapely polygon for desired area
pol = Polygon(study_area)

# get bounding box (minx, miny, maxx, maxy)
bbox = pol.bounds

Using `pystac_client` we can search the Planetary Computer's STAC endpoint for items matching our query parameters.

In [4]:
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = stac.search(
    bbox=bbox,
    datetime=date_range,
    collections=["sentinel-2-l2a"],
    limit=500,  # fetch items in batches of 500
    query={"eo:cloud_cover": {"lt": max_cloud}},
)

items = list(search.get_items())
print(f'Number of images is {len(items)}')

Number of images is 137


Depending on the year, this should return about 100-150 images for our study area over space, time, and cloudiness. Those items will still have *some* clouds over portions of the scenes, though. To create our cloudless mosaic, we'll load the data into an [xarray](https://xarray.pydata.org/en/stable/) DataArray using [stackstac](https://stackstac.readthedocs.io/) and then reduce the time-series of images down to a single image.

In [5]:
signed_items = []
for item in items:
    item.clear_links()
    signed_items.append(planetary_computer.sign(item).to_dict())

## Load Data

In this step we load the data and perform some initial cleaing that includes:
* subsetting to our exact bounding box
* removing pixels that correspond clouds and clouds shadows

To perform our cloud masking, we use Sentinel-2's Scene Classification Layer ([SCL](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm)) and mask out values 3, 8, 9, and 10.

In [6]:
data = (
    stackstac.stack(
        signed_items,
        assets=["B02","B03","B04","B08","SCL"],
        chunksize=4096,
        resolution=10,
        epsg=32760
    )
    .where(lambda x: x > 0, other=np.nan)  # sentinel-2 uses 0 as nodata
)

# Get bounding box in projection of data
project = pyproj.Transformer.from_crs(pyproj.CRS('EPSG:4326'), pyproj.CRS(data.crs), always_xy=True).transform
pol_utm = transform(project, pol)
minx, miny, maxx, maxy = pol_utm.bounds

# Subset data and mask clouds
data = data.sel(x=slice(minx, maxx), y=slice(maxy,miny))
data = data.where(~data.isel(band=4).isin([3,8,9,10])).isel(band=slice(0,4))
data

/srv/conda/envs/notebook/lib/python3.8/site-packages/stackstac/accumulate_metadata.py:151: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  props_arr = np.squeeze(np.array(props))


,Array,Chunk
Bytes,730.75 GiB,128.00 MiB
Shape,"(137, 4, 9260, 19328)","(1, 1, 4096, 4096)"
Count,153495 Tasks,8220 Chunks
Type,float64,numpy.ndarray


## Summarize Data

In this step we summarize the data by both band and by time. First we average the Red, Green, Blue and NIR bands to synthesize a panchromatic band, then compute the yearly median of the data. Depending on whether you're processing data from one year or multiple years, run the corresponding code.

### Synthesize the panchromatic band

In [7]:
pan = data.mean(dim='band')

### Single-year processing

In [8]:
grouped = pan.median(dim='time')

In [9]:
grouped = grouped.fillna(0)
grouped = grouped.rio.write_crs(pyproj.CRS(data.crs).to_string())

In [10]:
grouped = grouped.rename('Panchromatic')

In [11]:
grouped

<xarray.DataArray 'Panchromatic' (y: 9260, x: 19328)>
dask.array<where, shape=(9260, 19328), dtype=float64, chunksize=(463, 256), chunktype=numpy.ndarray>
Coordinates:
  * x                                        (x) float64 2.689e+05 ... 4.622e+05
  * y                                        (y) float64 5.647e+06 ... 5.554e+06
    instruments                              <U3 'msi'
    s2:product_type                          <U7 'S2MSI2A'
    s2:saturated_defective_pixel_percentage  float64 0.0
    s2:degraded_msi_data_percentage          float64 0.0
    sat:orbit_state                          <U10 'descending'
    constellation                            <U10 'Sentinel 2'
    s2:datatake_type                         <U8 'INS-NOBS'
    epsg                                     int64 32760
    spatial_ref                              int64 0

In [12]:
%%time
name = f's2_l2_{year}0601.tif'
print(name)
grouped.rio.to_raster(name)

s2_l2_20210601.tif
CPU times: user 23.2 s, sys: 4.53 s, total: 27.8 s
Wall time: 15min 56s


### Multi-year processing

In [ ]:
grouped = pan.groupby("time.year").median()
grouped = grouped.fillna(0)
grouped = grouped.rio.write_crs(pyproj.CRS(data.crs).to_string())
grouped = grouped.rename('Panchromatic')

In [ ]:
%%time
for y in grouped.year.to_numpy():
    grouped.sel(year=y).rio.to_raster(f's2_l2_{y}0601.tif')

### Mulit-year and season processing
This is generally not recommended for Sentinel-2 since it's difficult to create a truly cloud-free composite with seasonal data, but could be of use for some.

In [ ]:
import pandas as pd
year_season_idx = pd.MultiIndex.from_arrays([pan['time.year'].to_numpy(), pan['time.season'].to_numpy()])
pan.coords['year_season'] = ('time', year_season_idx)

In [ ]:
grouped = pan.groupby('year_season').median()

In [ ]:
grouped = grouped.rename('Panchromatic')
grouped = grouped.rio.write_crs(pyproj.CRS(data.crs).to_string())

In [ ]:
%%time
season_mid = {'DJF':'01','MAM':'04','JJA':'07','SON':'10'}

for i in grouped.year_season.to_numpy():
    y,s = i
    m = season_mid[s]
    out_name = f's2_l2_{y}{m}15.tif'
    grouped.sel(year_season=i).rio.to_raster(out_name)

## Close cluster
Once we're done with our processing, let's be a good steward of our resources and close our cluster

In [ ]:
cluster.close()

## Download Data
And you're done! The completed GeoTiff files should be in the same directory as this notebook, and can be downloaded via Jupyter's GUI